<a href="https://colab.research.google.com/github/Benteaux/karpathy-tutorials/blob/main/notebooks/makemore1Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**E01: Train a Trigram Language Model**

In [ ]:
# make a trigram - both counting & with neural nets

In [ ]:
names = open("names.txt", "r").read().splitlines()

In [ ]:
import torch

In [ ]:
N = torch.zeros(729, 27).int()

In [ ]:
alphabet = sorted(list(set('.'.join(names))))

In [ ]:
i = 0
stoi = {}
for let1 in (alphabet):
  for let2 in (alphabet):
    stoi[let1+let2] = i
    i += 1

In [ ]:
itos = {i:s for s, i in stoi.items}

In [ ]:
simpleStoi = {s:i for i,s in enumerate(alphabet)}
simpleItos = {i:s for s,i in simpleStoi.items()}

In [ ]:
count = 0
for word in names[:1]:
  word = ['.'] + ['.'] + list(word) + ['.'] # hallucinating 2 start tokens
  for ch1, ch2, ch3 in zip(word, word[1:], word[2:]):
    ix1 = stoi[ch1 + ch2]
    ix2 = simpleStoi[ch3]
    N[ix1, ix2] += 1
    count += 1

. e m
e m m
m m a
m a .


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (200, 200)) # probs not good figsize
plt.imshow(N, cmap = "Reds")
for i in range(729):
  for j in range(27):
    chstr = itos[i] + simpleItos[j]
    plt.text(j, i, chstr, ha = "center", va = "bottom", color = "black")
    plt.text(j, i, N[i][j].item(), ha = "center", va = "top", color = "black")

plt.axis("off")

In [ ]:
# create a probability distribution to sample from w/ model smoothing
P = (N + 1).float()
P /= P.sum(1, keepdims = True)

In [ ]:
P[1].sum()

tensor(1.0000)

In [ ]:
g = torch.Generator().manual_seed(3)
for i in range(10):
  i = 0
  simpleI = 0
  out = []
  while True:
    p = P[i]
    ix = torch.multinomial(p, num_samples = 1, replacement = True, generator = g).item()
    # ix can index along columns (27 possibilities)
    # must now take 2nd somehow and use it and the ix index to select a new probability distribution
    third = simpleItos[ix] # index of new character
    if third == '.':
      break
    second = itos[i][1] # should be the 2nd
    i = stoi[second + third]
    out.append(third)
  print(''.join(out))


cir
anah
releemaleeumana
olvi
siellier
lie
lychaus
riaileile
kott
kie


In [ ]:
# now need to calculate loss
# calculate average negative log likelihood for whole dataset
# take the sum of the logs of the probabilities it assigns to the stuff in the dataset, then negate it, then average it
negLogProb = 0
totalNames = 0
for name in names:
  totalNames += 1
  name = ['.'] + ['.'] + list(name) + ['.']
  for ch1, ch2, ch3 in zip(name, name[1:], name[2:]):
    ix1 = stoi[ch1 + ch2]
    ix2 = simpleStoi[ch3]
    prob = P[ix1][ix2]
    negLogProb -= torch.log(prob)

print(f'Average NLL = {negLogProb / totalNames}')

Average NLL = 15.754159927368164


In [ ]:
# Average NLL = 15.754159927368164

In [ ]:
# trigram is worse because it's held to a higher standard??? (best explanation I got, from chatGPT)

**E02: Split Dataset**

In [ ]:
len(names)

32033

In [ ]:
type(names)

list

In [ ]:
trainIndex = (int) (len(names) * 0.8)

In [ ]:
testIndex = trainIndex + (int) ((len(names) - trainIndex) / 2)

In [ ]:
import random

In [ ]:
print(names[0])
random.shuffle(names)
print(names[0])

tanitoluwa
kendrik


In [ ]:
trainNames = names[:trainIndex]
testNames = names[trainIndex:testIndex]
devNames = names[testIndex:]

In [ ]:
N = torch.zeros(729, 27)

In [ ]:
count = 0
stop = (int) (len(trainNames) * 0.6) - 1
for word in trainNames[:stop]:
  word = ['.'] + ['.'] + list(word) + ['.'] # hallucinating 2 start tokens
  for ch1, ch2, ch3 in zip(word, word[1:], word[2:]):
    ix1 = stoi[ch1 + ch2]
    ix2 = simpleStoi[ch3]
    N[ix1, ix2] += 1
    count += 1

In [ ]:
smoothing = 1
P = (N + smoothing).float()
P /= P.sum(1, keepdims = True)

In [ ]:
# OG Average NLL = 15.754159927368164

In [ ]:
negLogProb = 0
totalNames = 0
for name in devNames:
  totalNames += 1
  name = ['.'] + ['.'] + list(name) + ['.']
  for ch1, ch2, ch3 in zip(name, name[1:], name[2:]):
    ix1 = stoi[ch1 + ch2]
    ix2 = simpleStoi[ch3]
    prob = P[ix1][ix2]
    negLogProb -= torch.log(prob)

print(f'Average NLL = {negLogProb / totalNames}')

Average NLL = 16.166526794433594


In [ ]:
for word in trainNames[stop:]:
  word = ['.'] + ['.'] + list(word) + ['.'] # hallucinating 2 start tokens
  for ch1, ch2, ch3 in zip(word, word[1:], word[2:]):
    ix1 = stoi[ch1 + ch2]
    ix2 = simpleStoi[ch3]
    N[ix1, ix2] += 1
    count += 1
P = (N + smoothing).float()
P /= P.sum(1, keepdims = True)

In [ ]:
negLogProb = 0
totalNames = 0
for name in testNames:
  totalNames += 1
  name = ['.'] + ['.'] + list(name) + ['.']
  for ch1, ch2, ch3 in zip(name, name[1:], name[2:]):
    ix1 = stoi[ch1 + ch2]
    ix2 = simpleStoi[ch3]
    prob = P[ix1][ix2]
    negLogProb -= torch.log(prob)

print(f'Average NLL = {negLogProb / totalNames}')

Average NLL = 15.915894508361816


In [ ]:
# loss decreases slightly after completing training
# loss goes up slightly compared to whole dataset training. Implies overfitting to the training data
# unclear why it overfits when trained on less data - must not be overfitting

**E03: Smoothing Tuning**

In [ ]:
# smoothing = 1: 16.1248
# smoothing = 10: 17.406
# smoothing = 25: 18.4569
# smoothing = 50: 19.4234
# smoothing = 100: 20.4277
# smoothing = 250: 21.613
# smoothing = 500: 22.3040


# FOR TRIGRAM, MINIMUM SMOOTHING WAS BEST
# SINCE SMOOTHING MIRRORS REGULARIZATION, WOULD LIKELY HAVE SIMILAR RESULTS
# Makes sense since model wasn't overfitting